In [3]:
import faiss
import os
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

In [4]:
embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-large-en")

/home/lahirum/Documents/Mirada/RnD/FastApi/test2/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
#load vector store
vector_store=FAISS.load_local("vector_store",embeddings,allow_dangerous_deserialization=True)

In [6]:
question = "What is the total face value of the convertible note?"

In [7]:
#Maximum marginal relevance retrieval
retriever = vector_store.as_retriever(
    search_type="mmr"
)

In [8]:
retrieved_docs=retriever.invoke(question)
print(retrieved_docs)

[Document(id='3e138805-138c-4410-b78c-2d1f75ee707e', metadata={'source': './uploaded_pdfs/sample.txt'}, page_content='| Net cash from/(used in) investing activities                                              | 2,578,486                | (489,175)                    |\n| Cash flows from financing activities                                                      |                          |                              |\n| Proceeds from the issue of shares                                                         | -                        | 618,460                      |\n| Payment of share issue costs                                                              | -                        | (22,500)                     |\n| Payment of convertible note issue costs                                                   | (71,446)                 | -                            |\n| Proceeds from borrowings - convertible notes payable                                      | 500,000                

In [9]:
context="\\n".join([d.page_content for d in retrieved_docs])

In [10]:
context

"| Net cash from/(used in) investing activities                                              | 2,578,486                | (489,175)                    |\n| Cash flows from financing activities                                                      |                          |                              |\n| Proceeds from the issue of shares                                                         | -                        | 618,460                      |\n| Payment of share issue costs                                                              | -                        | (22,500)                     |\n| Payment of convertible note issue costs                                                   | (71,446)                 | -                            |\n| Proceeds from borrowings - convertible notes payable                                      | 500,000                  | 1,400,000                    |\n| Net cash from/(used in) financing activities                                   

In [ ]:
chat_template = [
        {"role": "system", "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer."""},
        {"role": "user", "content": f"""Context:
{context}
---
Now here is the question you need to answer.

Question: {question}"""},
    ]

In [ ]:
chat_template

In [ ]:
#Using an ollama model

from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(
        model="llama3.2", 
        messages=chat_template,
        think=False,
        options={
        "num_predict": 4096,
        "temperature": 0.6,
        "top_p": 0.95,
        "top_k": 20,
        "repeat_penalty": 1.05,
        }
    )
        


In [ ]:
response['message']['content']

Using ChatOllama

In [11]:
from langchain_core.prompts import ChatPromptTemplate

# Define a chat prompt template composed of system and user messages
chat_template= ChatPromptTemplate.from_messages([
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": f"""Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
])


In [12]:
from langchain_ollama import ChatOllama
llm= ChatOllama(
    model="llama3.2", 
    messages=chat_template,
    think=False,
    options={
        "num_predict": 4096,
        "temperature": 0.6,
        "top_p": 0.95,
        "top_k": 20,
        "repeat_penalty": 1.05,
    }    
)

In [ ]:
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser

# Define a RAG chain using a pipeline of runnables
# rag_chain=(
#     {"context":context,   # Combine retrieved docs
#      "question":RunnablePassthrough(),
#      }
#      |chat_template
#      |response['message']['content']
chain = chat_template|llm
     
     


In [14]:
answer=chain.invoke({"question": question})

In [16]:
answer

AIMessage(content='The total face value of the convertible note was $1,400,000.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-10-22T04:52:42.474378833Z', 'done': True, 'done_reason': 'stop', 'total_duration': 81436335216, 'load_duration': 1286598240, 'prompt_eval_count': 2012, 'prompt_eval_duration': 78073000000, 'eval_count': 17, 'eval_duration': 1550000000, 'model_name': 'llama3.2', 'model_provider': 'ollama'}, id='lc_run--19800c43-abeb-47cd-9530-b1b65a19d6b7-0', usage_metadata={'input_tokens': 2012, 'output_tokens': 17, 'total_tokens': 2029})

In [15]:
answer.content

'The total face value of the convertible note was $1,400,000.'

In [17]:
#Returning Multiple Values using Tuples
def multiple():
    operation = "Sum" 
    total = 5+10
    return operation, total;




In [18]:
operation, total = multiple()

In [19]:

print(operation, total)
#Output = Sum 15

Sum 15
